# Code for Sumo Enviroment

In [69]:
# Custom Environment
import gymnasium as gym
import numpy as np
import traci
import math
from os import path


class SumoEnv(gym.Env):
  def __init__(self):

    traci.start(["sumo", "-c", path.abspath("../SUMO/test.sumocfg")])

    ## SUMO VARIABLES ##
    self.bus_stop_positions = [
        [123, 974, 1872, 2764], [123, 827, 1742, 2702, 3592]]
    self.bus_locations = {"-overlap": "123", "-R2": "259", "-R1": "125",
                          "-R0": "267", "-L3": "117", "-L2": "110", "-L1": "123", "-L0": "120"}
    self.bus_ids = ["bus_r_0_0", "bus_r_0_1",
                    "bus_r_0_2", "bus_r_0_3", "bus_r_0_4"]

    # self.route_names = [["-overlap", "-R2", "-R1", "-R0"], ["-overlap", "-L3", "-L2", "-L1", "-L0"]]
    self.route_lengths = [3591, 4697]
    # self.route_junctions={"J1": ["-L0", "-R0", "-overlap"], "J2": ["-R0", "-R1"], "J3": ["-R1", "-R2"], "J4": ["-L3", "-R2", "-overlap"], "J5": ["-L0", "-L1"], "J7": ["-L1", "-L2"], "J8": ["-L2", "-L3"]}

    self.wait_time = 0
    self.delta_speed = 0.1
    self.min_speed_before_change = 30
    self.action_delta_speed = {
        0: (1-self.delta_speed), 2: (1+self.delta_speed)}

    ## GYM VARIABLES ##
    self.bus_num = 5
    bus_stops_num = 4
    bus_speed_max = 50

    # actions: [b1, b2 (...)] # each action is either 0 = slow down, 1 = keep speed, 2 = speed up
    self.action_space = gym.spaces.Box(low=np.array(
        [0]*self.bus_num), high=np.array([2]*self.bus_num), shape=(self.bus_num,), dtype=np.int32)

    # states: [avg_wait_time, b1_speed, b1_pos, b2_speed, b2_pos, (...),  bs1_pos, bs2_pos, bs3_pos, bs4_pos]
    wait_max = 100000.00
    low_obs = np.zeros([1 + 2*self.bus_num + bus_stops_num])
    high_obs = np.array([wait_max] + [bus_speed_max, self.route_lengths[0]]
                        * self.bus_num + [self.route_lengths[0]]*bus_stops_num)
    self.observation_space = gym.spaces.Box(low=low_obs, high=high_obs, shape=(
        1 + 2*self.bus_num + bus_stops_num,), dtype=np.float32)

    self.max_steps = 500
    self.current_step = 0

  def reset(self, seed=None):
      traci.close()
      self.wait_time = 0
      self.current_step = 0
      bus_stops_num = 4
      traci.start(["sumo", "-c", path.abspath("../SUMO/test.sumocfg")])
      
      # set the buses to the start of the route (MAY FAIL)
      return np.concatenate(([self.wait_time], np.zeros(2 * self.bus_num), self.bus_stop_positions[0])).astype(np.float32)[:15], {}


  def step(self, action):
    try:
      next_state = self.sumo_step()

      # set action for each bus: 0 = slow down, 1 = keep speed, 2 = speed up
      vehicles_length = len(traci.vehicle.getIDList())

      for i, bus_action in enumerate(action):
        if bus_action == 1 or i >= vehicles_length:
          break
        bus_id = self.bus_ids[i]
        bus_distance_driven = traci.vehicle.getDistance(bus_id)

        if np.sign(bus_distance_driven) == -1:
          break  # if bus hasnt driven yet, skip

        bus_route = traci.vehicle.getRouteID(bus_id)
        bus_position = round(bus_distance_driven % (
            self.route_lengths[0] if (bus_route == "r_0") else self.route_lengths[1]), 3)
        nearest_bus_stop_position = self._find_nearest(
            self.bus_stop_positions[0 if bus_route == "r_0" else 1], bus_position)
        bus_speed_km_t = traci.vehicle.getSpeed(bus_id)*3.6  # m/s to km/h

        interval = [-22, 3]
        # change speed if speed > min_speed_before_change and bus is not at a bus stop
        if bus_speed_km_t > self.min_speed_before_change and not (bus_position > nearest_bus_stop_position + interval[0] and bus_position < nearest_bus_stop_position + interval[1]):
          # speed is in m/s
          new_speed = self.action_delta_speed[bus_action] * \
              traci.vehicle.getSpeed(bus_id)
          # smoothly changes to new speed over 1 second
          traci.vehicle.slowDown(bus_id, new_speed, 1)

      # reward are given if the new waiting time is strictly lower, otherwise punished
      reward = 1 if next_state[0] < self.wait_time else -1

      # set the wait time to the current wait time
      self.wait_time = next_state[0]

      # check if done
      self.current_step += 1
      done = False
      if (self.current_step >= self.max_steps):
        done = True

      truncated = False
      return np.array(next_state, dtype=np.float32), reward, truncated, done, {}

    except Exception as e:  # if there is an error, close the simulation
      print("An error occurred. Closing simulation.")
      print("Error: ", e)
      traci.close()

  def render(self):
    pass

  def close(self):
    pass

  # SUMO FUNCTIONS
  def sumo_step(self):
    # state [avg_wait_time, b1_speed, b1_pos, b2_speed, b2_pos, (...),  bs1_pos, bs2_pos, bs3_pos, bs4_pos]
    new_state = [0] * (1 + 2 * self.bus_num) + self.bus_stop_positions[0]
    personsWaitingTimeList = []
    traci.simulationStep()

    vehicles = traci.vehicle.getIDList()
    persons = traci.person.getIDList()

    # finds the average waiting time
    for i in range(0, len(persons)):
      personWaitingTime = traci.person.getWaitingTime(persons[i])
      personsWaitingTimeList.append(personWaitingTime)

    persons_waiting_num = len(personsWaitingTimeList)
    new_state[0] = round(sum(personsWaitingTimeList) /
                         persons_waiting_num, 3) if persons_waiting_num > 0 else 0.0

    # finds bus speed and position
    for j in range(0, len(vehicles)):
      vehicleId = vehicles[j]
      if traci.vehicle.getRouteID(vehicleId) != "r_0":
        continue

      vehicleSpeed = traci.vehicle.getSpeed(vehicleId)*3.6  # m/s to km/h
      vehiclePosition = traci.vehicle.getDistance(vehicleId) % (self.route_lengths[0]
                                                                if (traci.vehicle.getRouteID(vehicleId) == "r_0") else self.route_lengths[1])
      new_state[1 + 2*j] = round(vehicleSpeed, 2)
      new_state[2 + 2*j] = round(vehiclePosition, 2)
    return new_state

  def _find_nearest(self, array, value):  # BUG FIND NEAREST BUS STOP
    idx = np.searchsorted(array, value, side="left")
    if (idx == len(array) and math.fabs(value - (array[0] + array[idx-1])) < math.fabs(value - array[idx-1])):
      return array[0]
    elif idx > 0 and math.fabs(value - array[idx-1]) < math.fabs(value - array[idx]):
      return array[idx-1]
    else:
      return array[idx]

#### Manual Testing

In [79]:
traci.close()

Step #5.00 (0ms ?*RT. ?UPS, TraCI: 12702ms, vehicles TOT 1 ACT 1 BUF 1)                   


In [80]:
# Test the environment
env = SumoEnv()

# Reset the environment to get the initial state
state, info = env.reset()

# Loop for multiple simulation steps
for _ in range(5):
  # Take an action (array of integers representing speed changes for each bus)
  # Example action (keep speed, slow down, speed up, ...)
  action = [1, 0, 2, 1, 0]

  # Perform a step in the environment
  next_state, reward, done, info, truncated = env.step(action)


  # Process the reward and next state
  np.set_printoptions(suppress=True)
  print(f"Step: {_}, Wait Time: {next_state[0]}, Reward: {reward}, Next State: {next_state}, Done: {done}, Info: {info} \n")

  # Check if the simulation is done
  if done:
    break

# Close the environment
env.close()

 Retrying in 1 seconds


Step #0.00 (0ms ?*RT. ?UPS, TraCI: 3ms, vehicles TOT 0 ACT 0 BUF 0)                      
 Retrying in 1 seconds


Step: 0, Wait Time: 0.0, Reward: -1, Next State: [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  123.
  974. 1872. 2764.], Done: False, Info: False 

Step: 1, Wait Time: 1.0, Reward: -1, Next State: [   1.      3.14    0.87    0.      0.      0.      0.      0.      0.
    0.      0.    123.    974.   1872.   2764.  ], Done: False, Info: False 

Step: 2, Wait Time: 0.6669999957084656, Reward: 1, Next State: [   0.667    7.03     2.83     0.       0.       0.       0.       0.
    0.       0.       0.     123.     974.    1872.    2764.   ], Done: False, Info: False 

Step: 3, Wait Time: 1.0, Reward: -1, Next State: [   1.      9.23    5.39    0.      0.      0.      0.      0.      0.
    0.      0.    123.    974.   1872.   2764.  ], Done: False, Info: False 

Step: 4, Wait Time: 1.6670000553131104, Reward: -1, Next State: [   1.667   11.54     8.6      0.       0.       0.       0.       0.
    0.       0.       0.     123.     974.    1872.    2764.   ], Done: Fa

## Stable baseline 3

### Checking Sumo Enviroment
Check that an environment follows Gym API. 
It also optionally check that the environment is compatible with Stable-Baselines.

In [81]:
traci.close()

Step #5.00 (0ms ?*RT. ?UPS, TraCI: 22127ms, vehicles TOT 1 ACT 1 BUF 1)                   


In [82]:
# Check that an environment follows Gym API
# It also optionally check that the environment is compatible with Stable-Baselines.
from stable_baselines3.common.env_checker import check_env

env = SumoEnv()
check_env(env, warn=True, skip_render_check=True)

 Retrying in 1 seconds


Step #0.00 (0ms ?*RT. ?UPS, TraCI: 6ms, vehicles TOT 0 ACT 0 BUF 0)                      
 Retrying in 1 seconds


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/stable_baselines3/common/env_checker.py:453: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf. https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/stable_baselines3/common/env_checker.py:464: UserWarning: Your action space has dtype int32, we recommend using np.float32 to avoid cast errors.
  warnings.warn(


Step #0.00 (0ms ?*RT. ?UPS, TraCI: 2ms, vehicles TOT 0 ACT 0 BUF 0)                      
 Retrying in 1 seconds


Step #1.00 (0ms ?*RT. ?UPS, TraCI: 32ms, vehicles TOT 1 ACT 1 BUF 1)                      
 Retrying in 1 seconds


# Stable baseline 3 PPO Implmentation